In [42]:
import os
import re
import shutil
import string

import matplotlib.pyplot as plt
import tensorflow as tf
from   tensorflow.keras import Sequential
from   tensorflow.keras import layers, losses, preprocessing
from   tensorflow.keras.layers import (
    Dense, Dropout, Embedding, GlobalAveragePooling1D as GAP)
from   tensorflow.keras.layers.experimental.preprocessing import (
    TextVectorization)

In [2]:
tf.__version__

'2.4.1'

In [3]:
DATA = '../../../../../data'

In [4]:
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
subdir = 'aclIMDB'
# BROKEN: requires certificate validation--downloading manually:
#dataset = tf.keras.utils.get_file(
#    'aclImdb_v1', url, untar=True, cache_dir=DATA, cache_subdir=subdir)
#dataset_dir = os.path.join(os.path.dirname(dataset), subdir)

In [5]:
dataset_dir = f'{DATA}/{subdir}'
os.listdir(dataset_dir)

['imdbEr.txt', 'test', 'imdb.vocab', 'README', 'train']

In [6]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['urls_unsup.txt',
 'neg',
 'urls_pos.txt',
 'urls_neg.txt',
 'pos',
 'unsupBow.feat',
 'labeledBow.feat']

In [7]:
sample_file = os.path.join(train_dir, 'pos/1181_9.txt')

with open(sample_file) as f:
    print(f.read())

Rachel Griffiths writes and directs this award winning short film. A heartwarming story about coping with grief and cherishing the memory of those we've loved and lost. Although, only 15 minutes long, Griffiths manages to capture so much emotion and truth onto film in the short space of time. Bud Tingwell gives a touching performance as Will, a widower struggling to cope with his wife's death. Will is confronted by the harsh reality of loneliness and helplessness as he proceeds to take care of Ruth's pet cow, Tulip. The film displays the grief and responsibility one feels for those they have loved and lost. Good cinematography, great direction, and superbly acted. It will bring tears to all those who have lost a loved one, and survived.


Using `text_dataset_from_directory` util, expects directory structure:
```
main_dir/
  class_a/
    a_text_1.txt
    a_text_2.txt
    ...
  class_b/
    b_text_1.txt
    ...
```

In [9]:
#remove_dir = os.path.join(train_dir, 'unsup')
#shutil.rmtree(remove_dir)

In [10]:
BATCH = 32
SEED = 1103

raw_training_ds = preprocessing.text_dataset_from_directory(
    train_dir, 
    batch_size=BATCH, 
    validation_split=0.2, 
    subset='training', 
    seed=SEED)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [11]:
for text_batch, label_batch in raw_training_ds.take(1):
    for i in range(3):
        print('Review:\n', text_batch.numpy()[i])
        print('Label:', label_batch.numpy()[i])

Review:
 b"Curly Sue is a 6 year old with an abundance of hair and a life as a drifter. She and her father, Bill (Jim Belushi), try to survive on the streets by being small time con artists. In Chicago, Bill decides to jump in front of a car in a pricey parking garage while Curly will scream about lawsuits and traction to the intended victim. It happens to be a very upscale lawyer named Grey (Kelly Lynch) who is appropriately appalled at what she has done. Not only do the scammers make some cash, they get to spend the night at Grey's plush apartment. Even then, Grey feels she owes them more so the three of them hang together for a spell. Grey only knows the lucrative law business and nothing about life. Who better to teach her than Bill and Curly, those savvy experts on life's realities? But, all good things must come to an end and there is no life for a legal expert and a couple of con men. Or is there? This is a sweet and funny movie about the unexpected. Curly is certainly as entert

In [12]:
for encoding in [0, 1]:
    print(f'Label {encoding} corresponds to '
          f'{raw_training_ds.class_names[encoding]}')

Label 0 corresponds to neg
Label 1 corresponds to pos


In [28]:
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    f'{DATA}/aclIMDB/train', batch_size=BATCH)

Found 25000 files belonging to 2 classes.


In [29]:
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    f'{DATA}/aclIMDB/test', batch_size=BATCH)

Found 25000 files belonging to 2 classes.


In [30]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(
        stripped_html, '[%s]' % re.escape(string.punctuation), '')

In [31]:
MAX_FEATURES = 10_000
SEQ_LEN = 250

In [32]:
vectorize_layer = TextVectorization(standardize=custom_standardization, 
                                    max_tokens=MAX_FEATURES,
                                    output_mode='int',
                                    output_sequence_length=SEQ_LEN)

In [33]:
# Make text-only dataset (no labes), then adapt()
train_text = raw_training_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [34]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [35]:
# Get batch of reviews and labels from data set
text_batch, label_batch = next(iter(raw_training_ds))
first_review, first_label = text_batch[0], label_batch[0]
print(f'Reveiw:\n{first_review}')
print('Label:', raw_training_ds.class_names[first_label])
print(f'Vectorized review:\n{vectorize_text(first_review, first_label)}')

Reveiw:
b'As a collector of movie memorabilia, I had to buy the movie poster for this film which, now that I\'ve finally seen it, has to be the best thing about it. There\'s nothing more attractive to hang on your wall than a 27x41 inch image of the melting man. However, there\'s nothing more awful to put in your VCR than an hour and a half long image of the melting man. At first I thought this movie was pure garbage but then I realized that it did have some qualities which made me laugh. The character of Dr. Ted Nelson has to be the most wishy-washy persona ever brought to the big screen. His dialogue is so trite it\'s unbelievable! ("It\'s incredible! He seems to be getting stronger as he melts!)<br /><br />And could somebody tell me please how the heck they know exactly how much time Steve has left before he melts completely and exactly what their plan is to "help" him? If this movie was meant to scare its audience, I think it missed its calling.'
Label: pos
Vectorized review:
(<tf.

In [36]:
for vec in [1694, 11, 89]:
    print(f'{vec} -> {vectorize_layer.get_vocabulary()[vec]}')
print('Vocab size:', len(vectorize_layer.get_vocabulary()))

1694 -> normally
11 -> i
89 -> dont
Vocab size: 10000


In [38]:
train_ds = raw_training_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [39]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [40]:
EMBED_DIM = 32

In [44]:
mod = Sequential([Embedding(MAX_FEATURES + 1, EMBED_DIM),
                  Dropout(0.2),
                  GAP(),
                  Dropout(0.2),
                  Dense(1)])
mod.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          320032    
_________________________________________________________________
dropout (Dropout)            (None, None, 32)          0         
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 320,065
Trainable params: 320,065
Non-trainable params: 0
_________________________________________________________________


In [46]:
mod.compile(loss=losses.BinaryCrossentropy(from_logits=True),
            optimizer='adam',
            metrics=tf.metrics.BinaryAccuracy(threshold=0.))

In [47]:
EPOCHS = 10
history = mod.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

Epoch 1/10
625/625 [==============================] - 7s 11ms/step - loss: 0.6753 - binary_accuracy: 0.6539 - val_loss: 0.5618 - val_binary_accuracy: 0.7992
Epoch 2/10
625/625 [==============================] - 2s 3ms/step - loss: 0.5212 - binary_accuracy: 0.8103 - val_loss: 0.4173 - val_binary_accuracy: 0.8560
Epoch 3/10
625/625 [==============================] - 2s 3ms/step - loss: 0.3947 - binary_accuracy: 0.8610 - val_loss: 0.3438 - val_binary_accuracy: 0.8757
Epoch 4/10
625/625 [==============================] - 2s 3ms/step - loss: 0.3289 - binary_accuracy: 0.8815 - val_loss: 0.3020 - val_binary_accuracy: 0.8890
Epoch 5/10
625/625 [==============================] - 2s 3ms/step - loss: 0.2880 - binary_accuracy: 0.8960 - val_loss: 0.2734 - val_binary_accuracy: 0.8980
Epoch 6/10
625/625 [==============================] - 2s 3ms/step - loss: 0.2576 - binary_accuracy: 0.9067 - val_loss: 0.2521 - val_binary_accuracy: 0.9055
Epoch 7/10
625/625 [==============================] - 2s 3ms/st